In [5]:
%pip install torch transformers accelerate pillow pdf2image
!apt-get install -y poppler-utils

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import warnings

# disable some warnings
transformers.logging.set_verbosity_error()
transformers.logging.disable_progress_bar()
warnings.filterwarnings('ignore')


device = 'cuda'  # or cpu
torch.set_default_device(device)

model = AutoModelForCausalLM.from_pretrained(
    'BAAI/Bunny-v1_0-4B',
    torch_dtype=torch.float16, # float32 for cpu
    device_map='auto',
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(
    'BAAI/Bunny-v1_0-4B',
    trust_remote_code=True)



In [6]:
from pdf2image import convert_from_path

pdf_path = '/home/Redacted.pdf'  
images = convert_from_path(pdf_path)

In [20]:
import torch
from PIL import Image

def limit_images(limit):
    limited_images=[]
    for i in range(0,len(images)-1):
        if(1<limit):
            limited_images.append(images[i])
        else:
            break
    return limited_images

limited_images=limit_images(230)
    
image = limited_images
image = image if isinstance(image, list) else [image]
prompt = 'what is the Assessment and Plan of the patient?'
text = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <image>\n{prompt} ASSISTANT:"

text_chunks = [tokenizer(chunk).input_ids for chunk in text.split('<image>')]

input_ids = torch.tensor(text_chunks[0] + [-200] + text_chunks[1][1:], dtype=torch.long).unsqueeze(0).to(device)


image_tensor = model.process_images(image, model.config).to(dtype=model.dtype, device=device)

output_ids = model.generate(
    input_ids,
    images=image_tensor,
    max_new_tokens=100,
    use_cache=True,
    repetition_penalty=1.0 # increase this to avoid chattering
)[0]

print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())

The assessment and plan of the patient is that they have a history of a neurological condition, and they have been referred for follow-up neurological consultation. The patient has experienced pre-operative symptoms, and the plan is to evaluate their condition further.
